In [90]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [91]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 1000.0; tt = 1000.0; tstep = 0.001; tspan = [0.0, t]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep);

In [92]:
index_hom = 1;

In [93]:
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461];

In [94]:
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
fp, _, _ = fixedpoints(ds, box, jacob_TM_);

In [95]:
ϵ_box = 1e-4
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v2 = normalize(v2, 2)
v3 = normalize(v3, 2)

v1t = transpose(v1)
v2t = transpose(v2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.998728    0.999899     0.0
 -0.0489879  -0.0141814    0.984697
  0.0119121   0.000705852  0.174274

In [96]:
N = 13
nrange = range(1, N - 1, step = 1)
dots = zeros(length(nrange), 3)

12×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [97]:
for (i, n) in enumerate(nrange)
    α1 = 0
    α2 = cos(2 * pi * n / N) * (ϵ_box / 2)
    α3 = sin(2 * pi * n / N) * (ϵ_box / 2)
    αs = [α1, α2, α3]
    dot = fp_ + A * αs
    dots[i, :] = dot
end

In [98]:
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf) - ϵ_box
end
affect!(integrator) = terminate!(integrator)

function condition(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end

condition (generic function with 2 methods)

In [99]:
indexn = 1
u01 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb1 = ContinuousCallback(condition, nothing, affect!)
prob1 = ODEProblem(TM, u01, tspan, p)
sol1 = solve(prob1, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb1)
norm1, linsolve1 = condition(sol1[end])

(0.00010000000000068929, [2.059025439264729e-7, 0.00010000000000068929, -3.7775436043480216e-5])

In [100]:
indexn = 2
u02 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb2 = ContinuousCallback(condition, nothing, affect!)
prob2 = ODEProblem(TM, u02, tspan, p)
sol2 = solve(prob2, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb2)
norm2, linsolve2 = condition(sol2[end])

(0.00010000000000112543, [6.452988456559679e-7, 0.00010000000000112543, -6.71116065856461e-5])

In [101]:
indexn = 3
u03 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb3 = ContinuousCallback(condition, nothing, affect!)
prob3 = ODEProblem(TM, u03, tspan, p)
sol3 = solve(prob3, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb3)
norm3, linsolve3 = condition(sol3[end])

(0.00010000000000316197, [9.395186055170412e-7, 0.00010000000000316197, -8.103695723871062e-5])

In [102]:
indexn = 4
u04 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb4 = ContinuousCallback(condition, nothing, affect!)
prob4 = ODEProblem(TM, u04, tspan, p)
sol4 = solve(prob4, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb4)
norm4, linsolve4 = condition(sol4[end])

(0.00010000000000295284, [8.350442538365155e-7, 0.00010000000000295284, -7.63803518874625e-5])

In [103]:
indexn = 5
u05 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb5 = ContinuousCallback(condition, nothing, affect!)
prob5 = ODEProblem(TM, u05, tspan, p)
sol5 = solve(prob5, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb5)
norm5, linsolve5 = condition(sol5[end])

(0.00010000000000147354, [4.218850202478422e-7, 0.00010000000000147354, -5.420181928389363e-5])

In [104]:
indexn = 6
u06 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb6 = ContinuousCallback(condition, nothing, affect!)
prob6 = ODEProblem(TM, u06, tspan, p)
sol6 = solve(prob6, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb6)
norm6, linsolve6 = condition(sol6[end])

(0.00010000000000204604, [5.6094879532632844e-8, 0.00010000000000204604, -1.955351654374828e-5])

In [105]:
indexn = 7
u07 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb7 = ContinuousCallback(condition, nothing, affect!)
prob7 = ODEProblem(TM, u07, tspan, p)
sol7 = solve(prob7, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb7)
norm7, linsolve7 = condition(sol7[end])

(0.00010000000000121698, [5.476305124252211e-8, -0.00010000000000121698, 1.9327322961506225e-5])

In [106]:
indexn = 8
u08 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb8 = ContinuousCallback(condition, nothing, affect!)
prob8 = ODEProblem(TM, u08, tspan, p)
sol8 = solve(prob8, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb8)
norm8, linsolve8 = condition(sol8[end])

(0.00010000000000058434, [4.1969434382892063e-7, -0.00010000000000058434, 5.410029695645017e-5])

In [107]:
indexn = 9
u09 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb9 = ContinuousCallback(condition, nothing, affect!)
prob9 = ODEProblem(TM, u09, tspan, p)
sol9 = solve(prob9, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb9)
norm9, linsolve9= condition(sol9[end])

(0.00010000000000232039, [8.350870543242375e-7, -0.00010000000000232039, 7.645449643420655e-5])

In [108]:
indexn = 10
u010 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb10 = ContinuousCallback(condition, nothing, affect!)
prob10 = ODEProblem(TM, u010, tspan, p)
sol10 = solve(prob10, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb10)
norm10, linsolve10= condition(sol10[end])

(0.00010000000000137189, [9.425803098323765e-7, -0.00010000000000137189, 8.124721304540177e-5])

In [109]:
indexn = 11
u011 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb11 = ContinuousCallback(condition, nothing, affect!)
prob11 = ODEProblem(TM, u011, tspan, p)
sol11 = solve(prob11, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb11)
norm11, linsolve11= condition(sol11[end])

(0.00010000000000103292, [6.493724784436616e-7, -0.00010000000000103292, 6.737426699910598e-5])

In [110]:
indexn = 12
u012 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb12 = ContinuousCallback(condition, nothing, affect!)
prob12 = ODEProblem(TM, u012, tspan, p)
sol12 = solve(prob12, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb12)
norm12, linsolve12= condition(sol12[end])

(0.00010000000000074293, [2.0855325794699174e-7, -0.00010000000000074293, 3.803084170101343e-5])

In [ ]:
ts, tf = 1, length(sol)
idx, idy, idz = 2, 3, 1

f = Figure(resolution = (700, 700))

ax = LScene(f[1, 1], show_axis = true)
    
scale!(ax.scene, 50, 50, 1)

lines!(sol1[idx, ts: length(sol1)], sol1[idy, ts: length(sol1)], sol1[idz, ts: length(sol1)], linewidth = 1.0, color = :deeppink)
lines!(sol2[idx, ts: length(sol2)], sol2[idy, ts: length(sol2)], sol2[idz, ts: length(sol2)], linewidth = 1.0, color = :deeppink)
lines!(sol3[idx, ts: length(sol3)], sol3[idy, ts: length(sol3)], sol3[idz, ts: length(sol3)], linewidth = 1.0, color = :deeppink)
lines!(sol4[idx, ts: length(sol4)], sol4[idy, ts: length(sol4)], sol4[idz, ts: length(sol4)], linewidth = 1.0, color = :deeppink)
lines!(sol5[idx, ts: length(sol5)], sol5[idy, ts: length(sol5)], sol5[idz, ts: length(sol5)], linewidth = 1.0, color = :deeppink)
lines!(sol6[idx, ts: length(sol6)], sol6[idy, ts: length(sol6)], sol6[idz, ts: length(sol6)], linewidth = 1.0, color = :deeppink)
lines!(sol7[idx, ts: length(sol7)], sol7[idy, ts: length(sol7)], sol7[idz, ts: length(sol7)], linewidth = 1.0, color = :deeppink)


scatter!(ax, fp[1][idx], fp[1][idy], fp[1][idz], color = :blue)
scatter!(ax, dots[:, idx], dots[:, idy], dots[:, idz], markersize = 5)


display(GLMakie.Screen(), f);

In [112]:
linsolve1[idx]

2.059025439264729e-7

In [115]:
f = Figure(resolution = (700, 700))

ax = Axis(f[1, 1])
idx, idy = 1, 3

scatter!(ax, 0.0, 0.0, color = :green, markersize = 15, marker = :rect)

scatter!(ax, linsolve1[idx], linsolve1[idy], color = :black, markersize = 5)
scatter!(ax, linsolve2[idx], linsolve2[idy], color = :black, markersize = 5)
scatter!(ax, linsolve3[idx], linsolve3[idy], color = :black, markersize = 5)
scatter!(ax, linsolve4[idx], linsolve4[idy], color = :black, markersize = 5)
scatter!(ax, linsolve5[idx], linsolve5[idy], color = :black, markersize = 5)
scatter!(ax, linsolve6[idx], linsolve6[idy], color = :black, markersize = 5)
scatter!(ax, linsolve7[idx], linsolve7[idy], color = :black, markersize = 5)
scatter!(ax, linsolve8[idx], linsolve8[idy], color = :black, markersize = 5)
scatter!(ax, linsolve9[idx], linsolve9[idy], color = :black, markersize = 5)
scatter!(ax, linsolve10[idx], linsolve10[idy], color = :black, markersize = 5)
scatter!(ax, linsolve11[idx], linsolve11[idy], color = :black, markersize = 5)
scatter!(ax, linsolve12[idx], linsolve12[idy], color = :black, markersize = 5)

display(GLMakie.Screen(), f);